In [1]:
import os
import pandas as pd

In [2]:
org_path =  '/content/drive/MyDrive/soil'

In [3]:
soil_name = os.listdir(org_path)

In [4]:
soil_name

['Sandy_soil', 'Loamy_soil', 'Black_soil', 'Clay_soil', 'Red_soil']

In [5]:
path = []
label = []

In [6]:
for dis in soil_name:
  for i in os.listdir(os.path.join(org_path, dis)):
    path.append(os.path.join(org_path, dis, i))
    label.append(dis)

In [7]:
df = pd.DataFrame(zip(path, label))

In [8]:
df.columns = ['path', 'label']

In [9]:
df.columns = ['path', 'label']

In [10]:
import torchvision
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
from torchvision.models import vgg16
from sklearn.model_selection import train_test_split
import torch.nn.functional as Fun

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.path, df.label, test_size=0.15, random_state=42)

In [12]:
df = pd.concat([X_train,y_train], axis=1).reset_index(drop=True)
df_test = pd.concat([X_test,y_test], axis=1).reset_index(drop=True)

In [13]:
df.label.value_counts()

Black_soil    80
Loamy_soil    79
Red_soil      75
Sandy_soil    72
Clay_soil     69
Name: label, dtype: int64

In [14]:
df_test.label.value_counts()

Sandy_soil    19
Red_soil      16
Clay_soil     12
Loamy_soil    11
Black_soil     9
Name: label, dtype: int64

In [15]:
lb = LabelEncoder()
df_test.label =  lb.fit_transform(df_test.label)
df.label =  lb.fit_transform(df.label)
dict(zip(lb.classes_, lb.transform(lb.classes_)))

{'Black_soil': 0,
 'Clay_soil': 1,
 'Loamy_soil': 2,
 'Red_soil': 3,
 'Sandy_soil': 4}

In [16]:
potato_classes = dict(zip(lb.classes_, lb.transform(lb.classes_)))
potato_classes

{'Black_soil': 0,
 'Clay_soil': 1,
 'Loamy_soil': 2,
 'Red_soil': 3,
 'Sandy_soil': 4}

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [18]:
class SoilDataset(Dataset):
    def __init__(self, df):
        self.fpath = df.path
        self.label = df.label
        self.transforms = T.Compose(
        [T.ToTensor(),
        T.Normalize(mean=[0.485,
           0.456, 0.406],std=[0.229, 0.224, 0.225])]
        )
    def __len__(self):
        return len(self.fpath)
    
    def __getitem__(self,ix):
        f = self.fpath[ix]
        label = self.label[ix]
        im = cv2.imread(f)
        im = cv2.resize(im, (224,224))
        # im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im = self.transforms(im)
        
        return torch.tensor(im,dtype=torch.float).to(device), torch.tensor(label, dtype=torch.float).long().to(device)

In [19]:
def get_model():
    model = torchvision.models.vgg16(pretrained=True)
    for param in model.features.parameters():
                   param.requires_grad = False
    model.avgpool = model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.classifier = nn.Sequential(nn.Flatten(),
                                   nn.Linear(512, 128),
                                   nn.ReLU(),
                                   nn.Dropout(0.2),
                                   nn.Linear(128, 5),
                                   nn.Softmax())
    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

In [20]:
model, loss_fn, optimizer = get_model()
from torchsummary import summary

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [21]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [22]:
def get_data():
    train = SoilDataset(df)
    test = SoilDataset(df_test)
    trn_dl = DataLoader(train, batch_size=16,shuffle=True, drop_last=True)
    test_dl = DataLoader(test, batch_size=16, shuffle=True, drop_last=True)
    return trn_dl, test_dl

In [23]:
def train_batch(x, y, model, loss_fn, opt):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [24]:
@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    pred = model(x)
    _, is_correct = torch.max(pred.data,1)
    acc_all = (is_correct==y).detach().cpu().numpy()
    return acc_all

In [25]:
@torch.no_grad()
def val_loss_trn(x, y, model, loss_fn):
  prediction = model(x)
  val_loss = loss_fn(prediction, y)
  return val_loss.item()

In [26]:
trn_ldr, test_ldr = get_data()


In [27]:
import cv2

In [28]:
train_loss = []
train_accuracies= []
val_loss = [] 
val_accuracies = []

for i in range(20):
    print(f'Epoch: _________*****{i}*****_______')
    train_epoch_losses, train_epoch_accuracies = [], []
    val_epoch_accuracies, val_epoch_losses = [], []



    for ix, batch in (enumerate(iter(trn_ldr))):
        
        x, y = batch
        # x = x.permute(0, 3,1,2)
        train_epoch_losses.append(train_batch(x, y, model, loss_fn, optimizer))
    train_epoch_loss = np.array(train_epoch_losses).mean()
    print(f'Epoch: _________*****{i} Training Loss : {train_epoch_loss} *****_______')



    for ix, batch in (enumerate(iter(trn_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        train_epoch_accuracies.append(sum(accuracy(x, y, model)) / len(y))
    train_epoch_accuracy = np.array(train_epoch_accuracies).mean()
    print(f'Epoch: _________*****{i} Training Accuracy: {train_epoch_accuracy} *****_______')


    for ix, batch in (enumerate(iter(test_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        val_epoch_losses.append(val_loss_trn(x, y, model, loss_fn))
    val_epoch_loss = np.array(val_epoch_losses).mean()
    print(f'Epoch: _________*****{i} Validation Loss : {val_epoch_loss} *****_______')

    for ix, batch in (enumerate(iter(test_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        val_epoch_accuracies.append(sum(accuracy(x, y, model)) / len(y))
    val_epoch_accuracy = np.array(val_epoch_accuracies).mean()
    print(f'Epoch: _________*****{i} Validation Accuracy. {val_epoch_accuracy} *****_______')
    print('\n')

    train_loss.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_accuracies.append(val_epoch_accuracy)
    val_loss.append(val_epoch_loss)

Epoch: _________*****0*****_______


<ipython-input-18-73c877ca0024>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(im,dtype=torch.float).to(device), torch.tensor(label, dtype=torch.float).long().to(device)
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: _________*****0 Training Loss : 1.6005148887634277 *****_______
Epoch: _________*****0 Training Accuracy: 0.3804347826086957 *****_______
Epoch: _________*****0 Validation Loss : 1.576924741268158 *****_______
Epoch: _________*****0 Validation Accuracy. 0.296875 *****_______


Epoch: _________*****1*****_______
Epoch: _________*****1 Training Loss : 1.541486760844355 *****_______
Epoch: _________*****1 Training Accuracy: 0.48641304347826086 *****_______
Epoch: _________*****1 Validation Loss : 1.5352149307727814 *****_______
Epoch: _________*****1 Validation Accuracy. 0.421875 *****_______


Epoch: _________*****2*****_______
Epoch: _________*****2 Training Loss : 1.4807667006617007 *****_______
Epoch: _________*****2 Training Accuracy: 0.5869565217391305 *****_______
Epoch: _________*****2 Validation Loss : 1.4980214536190033 *****_______
Epoch: _________*****2 Validation Accuracy. 0.46875 *****_______


Epoch: _________*****3*****_______
Epoch: _________*****3 Training Loss : 

In [31]:
torch.save(model.to('cpu').state_dict(), '/content/drive/MyDrive/soil.pth')

In [ ]:
transforms = T.Compose(
        [T.ToTensor(),
        T.Normalize(mean=[0.485,
           0.456, 0.406],std=[0.229, 0.224, 0.225])]
        )

    
def read_transform(img):
        f = img
        im = cv2.imread(f)
        im = cv2.resize(im, (224,224))
        im = transforms(im)
        return torch.tensor(im,dtype=torch.float).to(device)

def get_model(no_classes):
    model = torchvision.models.vgg16(pretrained=True)
    for param in model.features.parameters():
                   param.requires_grad = False
    model.avgpool = model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.classifier = nn.Sequential(nn.Flatten(),
                                   nn.Linear(512, 128),
                                   nn.ReLU(),
                                   nn.Dropout(0.2),
                                   nn.Linear(128, no_classes),
                                   nn.Sigmoid())

    return model.to(device)

def load_model(model_path, classes):
  model = get_model(classes)
  state_dict = torch.load(model_path)
  return model,state_dict

In [ ]:
def predict(image, crop):
  img = read_transform(image)
  img = img.to(device)
  model, state_dict = load_model('/content/potato.pth', 4)
  model.load_state_dict(state_dict)
  output = model(img.unsqueeze_(0))
  pred, conf = output.max(-1)
  
  return conf.item(), pred

In [32]:
import sklearn 
print(sklearn.__version__)

1.0.2
